In [ ]:
%pip install transformers torch clickhouse_connect accelerate

In [ ]:
import clickhouse_connect
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def txt2embeddings(text: str, tokenizer, model, device="cpu"):
    encoded_input = tokenizer(
        [text],
        padding=True,
        return_tensors="pt",
        truncation=True,
        max_length=512,
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    with torch.no_grad():
        model_output = model(**encoded_input)

    return mean_pooling(model_output, encoded_input["attention_mask"])


def load_models(model):
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModel.from_pretrained(model)
    return tokenizer, model

In [ ]:
TABLE_NAME = "SbertEmb"
MODEL_NAME = "ai-forever/sbert_large_nlu_ru"
HOST = ""
PORT = "80"

In [2]:
text = """От Об отмене Методических рекомендаций Банка России по усилению контроля операторами по переводу денежных средств за деятельностью банковских платежных агентов от 14 04 2016 11-МР 
С даты издания настоящего письма Банка России отменяются Методические рекомендации Банка России по усилению контроля операторами по переводу денежных средств за деятельностью банковских платежных агентов от 14 04 2016 11-МР 
Настоящее письмо Банка России подлежит опубликованию в Вестнике Банка России и размещению на официальном сайте Банка России в информационно-телекоммуникационной сети Интернет 
Первый заместитель Председателя Банка России О Н Скоробогатова""" #String
title = "Об отмене Методических рекомендаций Банка России по усилению контроля операторами по переводу денежных средств за деятельностью банковских платежных агентов от 14.04.2016 11-МР"
url = "https://cbr.ru/Crosscut/LawActs/File/7673" #String
date = "от 11.01.2024" #String
number = "04-45/107" #String

In [ ]:
tokenizer, model = load_models(MODEL_NAME)

In [ ]:
embeddings = txt2embeddings(text, model, tokenizer)

In [ ]:
client = clickhouse_connect.get_client(host=HOST, port=PORT)
print("Ping:", client.ping())

In [ ]:
values = f"('{title}','{url}','{date}','{number}','{text}','[{embeddings}]'),"
query = f"""INSERT INTO "{TABLE_NAME}"("Name","Url","Date","Number", "Text", "Embedding") VALUES {values}"""
client.command(query)